This is a rule-based stemmer. The idea is I get some rules from a grammar book and then add them in here.
First, let's get the data from FB.

In [3]:
import os
import xml.etree.ElementTree as ET

corpora_dir = "../FormosanBank/Corpora" # this should be the relative path to your FormosanBank download
FIND_LANG = "pyu" # Puyuma
FIND_GLOTTO = "nanw1237" # nanwang
FIND_DIALECT = "Nanwang" # Nanwang
consonant_list = ['lr','ng','tr','dr','m','n','p','t','k','b','d','g','s','r','l','y','w'] # used for infixes

If we want to be really sure, there are some other rules we can add in to check the dialect. For instance, in Puyuma if the letter 'b' or the digram 'dr' is present then it's Nanwang (barring loanwords).

In [4]:
def get_all_xmls():
# gets all .xml files in corpora_dir.
    all_xmls = []
    for root, dirname, filenames in os.walk(corpora_dir):
        for f in filenames:
            if f.endswith("xml"):
                all_xmls.append(os.path.join(root,f))
    return all_xmls

In [5]:
# takes in a list of xml files and finds which ones match our desired language code(s).
# See the formosanbank gitbook for more explanation about the xml format:
# https://ai4commsci.gitbook.io/formosanbank/the-bank-architecture/formosanbank-xml-format
def get_lang_xmls(file_list, match_lang=FIND_LANG, match_glotto=FIND_GLOTTO, match_dialect=FIND_DIALECT) -> list[str]:
    lang_xmls = []
    print(f"Finding xml files with language code {match_lang}, glotto code {match_glotto}, dialect {match_dialect}")
    for filepath in file_list:
        tree = ET.parse(filepath)
        root = tree.getroot()
        if root == None:
            print(f"Unable to parse file: {filepath}")
        # taken from formosanbank validate_xml.py
        lang = root.get("{http://www.w3.org/XML/1998/namespace}lang")
        if not lang:
            # print(f"{filepath} doesn't appear to have a [lang] attrib: {root.attrib}")
            continue
        glottocode = root.get("glottocode")
        dialect = root.get("dialect")
        if lang.lower() == match_lang.lower():
            if not glottocode and not dialect: # If no glotto or dialect, but language matches, add it
                # print(f"glotto: {glottocode} | dialect: {dialect} | file: {' '.join(filepath.split('/')[-5:])}")
                # we assume that just the language code is enough
                lang_xmls.append(filepath)
            else:
                # If glottocode or dialect match, add it
                if (glottocode and match_glotto and glottocode.lower() == match_glotto.lower()) or (dialect and match_dialect and dialect.lower() == match_dialect.lower()):
                        lang_xmls.append(filepath)
    # print(f"Found language codes: {str(list(set(found_langs)))}")
    # print(f"Found dialects of {match_lang}: {str(list(set(found_dialects)))}")
    print(f"Found {len(lang_xmls)} matching xml files")
    if len(lang_xmls) < 6:
        for x in lang_xmls:
            print('\t '.join(x.split('/')[3:]))
    return lang_xmls

In [6]:
# takes in an xml root, finds all 'sentence' elements, and returns a list of the 'form (standard)' element's text.
# See the formosanbank gitbook for more explanation. 
def get_sent_list(root) -> list[str]:
    sents = root.findall(".//S")
    texts = []
    for s in sents:
        form_children = []
        for child in s:
            if child.tag == "FORM":
                form_children.append(child)
            # there is a 'standard' and 'original' form for many sentences. 
            # If there's only one found, then add the sentence.
            # Otherwise, add the 'standard' form.
            if len(form_children) == 1:
                texts.append(form_children[0].text)
            else:
                for child in form_children:
                    kind = child.get("kindOf")
                    if kind == "standard":
                        texts.append(child.text)
    return texts

Now we have defined our methods, but we haven't used them yet. Let's get all the XML files in the corpora directory, and then filter for our desired language and dialect.

In [7]:
all_xmls = get_all_xmls()
print(len(all_xmls))

17108


In [8]:
lang_xmls = get_lang_xmls(all_xmls)

Finding xml files with language code pyu, glotto code nanw1237, dialect Nanwang
Found 18 matching xml files


We found 18 xml files out of 17,108 in our corpora directory. (Keep in mind that not all of the 17k XML files are language resources.) Now we parse each of these files and get all of the 'sentence' elements in them. We check how many sentences we got, and how many are unique, by printing out the length of the list and set respectively.

In [9]:
sent_list = []
for x in lang_xmls:
    root = ET.parse(x).getroot()
    x_list = get_sent_list(root)
    sent_list += x_list

print(len(sent_list)) # how many sentences total for our language
print(len(set(sent_list))) # how many unique sentences for our language

84351
27078


The difference in the list of all sentences and the 'set' of unique sentences is likely due to many dictionary definitions (i.e., single-word sentences) being included across various dictionaries. Also, it appears some of the ILRDF resources have some sentences repeated across different learning units.

Next, we want to collect all the words and sterilize them. We split the sentences into words, and then set the word to lowercase and strip off extra punctuation. The `?` glyph is used as a glottal stop in some languages in FormosanBank (more consistent for displaying instead of the IPA glyph) and the `'` glyph is also used in some languages. For Nanwang, neither are currently used in the orthography, despite glottal stops existing in the language.

In [10]:
corpus = []
bad_sents = []
for s in sent_list:
    # If the sentence is empty, then move on
    if not s:
        bad_sents.append(s)
        continue
    words = s.split()
    for w in words:
        w_clean = w.strip(' ,.!"`~![](){}|/\\<>#$@%^&*_-=+?').lower()
        if w_clean != '':
            # Some words have an apostrophe. Skip them for now.
            if w.find("'") != -1:
                continue
            # It appears that sometimes there's a spacing issue, and two words get glued together by a question mark.
            # If so, we split it into two words and then add them both.
            if w.find('?') != -1:
                fixed_w_list = w.split('?')
                for fw in fixed_w_list:
                    corpus.append(fw)
            else:
                corpus.append(w_clean)
print(len(corpus))
print(f"Found {str(len(bad_sents))} non-sentences (blank) out of {str(len(sent_list))}")

634847
Found 48 non-sentences (blank) out of 84351


In [11]:
all_words = set(corpus) # our 'dict' of words we've seen
print(len(all_words))

17733


So far we've read in the corpus for our language, and gotten all the sentences and words. We have a 'dictionary' of valid words (i.e., words that exist in our corpus) that contains 18,622 unique words. If we want to measure word frequency we can use our list of all sentences (or even unique sentences) to calculate.

Up until now our methodology has simply been 'reading in the corpus', which isn't very exciting, and is the exact same for any language tool (autocorrect, spell check, stemmer, word prediction, etc). 

We want to make a word stemmer, so now that we have all our language resources loaded in, we can start by reading the `rules.json` file that defines how words are conjugated in Puyuma.

In [12]:
import json
rules_file = "rules.json"

with open(rules_file, 'r') as f:
    rule_data = json.load(f)

for top_level in rule_data:
    print(top_level)

simple affix
compound affix
reduplication


At the highest level, we have simple affixes, compound affixes, and reduplication. Thankfully these are well-organized, so they're not too hard to parse. Let's start with the simple affixes.

In [13]:
print('Simple infixes:')
infixes = []
prefixes = []
suffixes = []
for rule in rule_data['simple affix']['infix']:
    infixes.append(rule['orthography'].strip('-'))
print(infixes)

print('=============')
print('Simple prefixes:')
for rule in rule_data['simple affix']['prefix']:
    prefixes.append(rule['orthography'].strip('-'))
print(prefixes)

print('=============')
print('Simple suffixes:')
for rule in rule_data['simple affix']['suffix']:
    suffixes.append(rule['orthography'].strip('-'))
print(suffixes)


Simple infixes:
['in', 'em', 'en', 'um', 'un', 'im']
Simple prefixes:
['be', 'i', 'ku', 'pa', 'ra', 'be', 're', 'ru', 'sa', 'ta', 'ti', 'tu', 'u', 'kan', 'ka', 'ki', 'ma', 'me', 'mi', 'mu', 'ni', 'pa', 'pi', 'pu']
Simple suffixes:
['an', 'aw', 'ay', 'anay', 'u', 'i']


If need be we can print out the 'rule' part of the various affix rule objects, but we already know how simple prefixes and suffixes work, and by reading `rules_json` we also know that an infix only occurs after the first consonant.

Below I've written some helper functions to check if a given string matches the given rule, as well as a 'stemming' function if it matches. These are mapped in the `rules_to_methods_dict`, if we wanted to add another rule-based stemming method, all we would need to define is the list of affixes, the way to check if the input string and input affix match, and the way to replace the affix if found.

In `get_word_stems` we go through an input list of `candidate_words` and check the appropriate affixes and rules based on the passed in `rule`. If the 'stemmed' word is in our list of `all_words`, then we add it to our output dict as a key, and any words which map to that stem as its values.

**Note:** We are assuming here that the stem word exists as its own word. This is not always true across languages, for instance Atayalic languages have a default conjugation that is present if no others are used.

In [34]:
# wrapper function to have everything play nicely in get_word_stems
# All this does is replace the *last* instance of a substring in the input string
# Used for stemming a suffix
def reverse_replace(input_word, string_to_replace, replace_with, max_replacements):
    return replace_with.join(input_word.rsplit(string_to_replace, max_replacements))

# Wrapper function to check if the given word has the given infix in the appropriate spot
def hasinfix(word_to_check, infix):
    for c in consonant_list:
        if word_to_check.startswith(c) and word_to_check.replace(c,'',1).startswith(infix):
            return True
    return False

rule_to_methods_dict = {
    'prefix': {'affix_list': prefixes, 'check_function' : str.startswith},
    'suffix': {'affix_list': suffixes, 'check_function' : str.endswith, 'replace_function': reverse_replace},
    'infix':  {'affix_list': infixes,  'check_function': str.startswith},
}

# Takes in a list of candidate words and the corresponding rule
# Sets the 'stemming function' (replace_func) and affix_list based on which rule we are looking at
# Returns a dictionary of {stem: [words, which, map, to, stem] ...}
def get_word_stems(candidate_words, rule):
    stem_affix_dict = {}
    affix_list = rule_to_methods_dict[rule]['affix_list']
    check_function = rule_to_methods_dict[rule]['check_function']
    replace_function = rule_to_methods_dict[rule].get('replace_function', str.replace)
        
    for cand in candidate_words:
        for affix in affix_list:
            if check_function(cand, affix):
                stem = replace_function(cand, affix, '', 1)
                if stem in all_words:
                    if stem not in stem_affix_dict:
                        stem_affix_dict[stem] =[]
                    if cand not in stem_affix_dict[stem]:
                        stem_affix_dict[stem].append(cand)

    return stem_affix_dict

# Finally, we need a helper function that will take two dictionaries, and return a dictionary that is a combination of the two
# We write our own function because we need the values which are lists to be joined, not overwritten
def merge_two_dicts(base_dict, update_dict):
    ret_dict = base_dict
    for k in update_dict.keys():
        if k in base_dict.keys():
            vals = set(base_dict[k])
            vals.update(update_dict[k])
            ret_dict[k] = list(vals)
        else:
            ret_dict[k] = update_dict[k]
    return ret_dict

# returns the first x keys and values of dict as a string
# if x is not given, returns the entire dict
def get_x_dict_entries(input_dict, x=-1):
    i = 0
    ret_str = ""
    for k in input_dict.keys():
        ret_str += f"Stem: {k}"
        ret_str += "\n"
        ret_str += f"Base words: {input_dict[k]}"
        ret_str += "\n\n"
        i += 1
        if i == x:
            break
    print(f"Got {i} entries from input dict")
    return ret_str

With these functions defined, we can now do multiple passes. This is because some words might have a prefix, suffix, and infix, or it could be even more complex with an infix in the middle of a prefix. The methodology is essentially:
1. Get 1-step stems from all_words
2. For all the 1-step stems, check if they can be 'stemmed' again based on the above rules
3. If so, then create a dict of 2-step stems using the 1-step stems as input
4. Repeat until we no longer find any new 'stems'
5. Once we stop finding new 'stems', we remove all the keys (stems) which occur as values (base words)

With all our helper functions defined, we will execute this process in code below.

In [35]:
# This will be the dictionary that we continually update with new stems and their base words
all_stems_to_bases_dict = {}

# This will be the list of all base words. It is useful to have as its own continuous list so that we can iterate through it later
all_base_words = set()

# To begin, we need to check the '0-step stems', i.e. all the words in our corpus
words_to_check = all_words

print(f"Starting the iterative loop with {len(all_words)} words to check")
i = 0
while len(words_to_check) != 0:
    i += 1
    if i >= 100:
        print("Uh oh, did you make an infinite loop?")
        break
    # This stores the k-step stems and their base words for each step
    stem_base_update_dict = {}
    for r in rule_to_methods_dict.keys():
        rule_candidate_dict = get_word_stems(words_to_check, r)
        print(f"Found {len(rule_candidate_dict)} candidate words for rule '{r}'")
        stem_base_update_dict = merge_two_dicts(stem_base_update_dict, rule_candidate_dict)
    print(f"Found a total of {len(stem_base_update_dict)} {i}-step stems")
    
    all_stems_to_bases_dict = merge_two_dicts(all_stems_to_bases_dict, stem_base_update_dict)
    for k in stem_base_update_dict.keys():
        all_base_words.update(stem_base_update_dict[k])
    words_to_check = stem_base_update_dict.keys()
    print(f"Now there are {len(words_to_check)} words to check after iteration: {i}")
    print("") # newline

print(f"There is a total of {len(all_stems_to_bases_dict)} potential stems")
print(f"There is a total of {len(all_base_words)} base words to make those stems")
print("")
print("Removing all base words from the stem dict")
final_stem_dict = {}
for k in all_stems_to_bases_dict:
    if k not in all_base_words:
        final_stem_dict[k] = all_stems_to_bases_dict[k]

print(f"Final stem dict has a length of {len(final_stem_dict)}")
print(f"First {amount_to_print} entries:")
print_str = get_x_dict_entries(final_stem_dict, amount_to_print)
print(print_str)

Starting the iterative loop with 17733 words to check
Found 1877 candidate words for rule 'prefix'
Found 1865 candidate words for rule 'suffix'
Found 134 candidate words for rule 'infix'
Found a total of 3194 1-step stems
Now there are 3194 words to check after iteration: 1

Found 332 candidate words for rule 'prefix'
Found 396 candidate words for rule 'suffix'
Found 30 candidate words for rule 'infix'
Found a total of 631 2-step stems
Now there are 631 words to check after iteration: 2

Found 59 candidate words for rule 'prefix'
Found 35 candidate words for rule 'suffix'
Found 7 candidate words for rule 'infix'
Found a total of 86 3-step stems
Now there are 86 words to check after iteration: 3

Found 4 candidate words for rule 'prefix'
Found 3 candidate words for rule 'suffix'
Found 1 candidate words for rule 'infix'
Found a total of 6 4-step stems
Now there are 6 words to check after iteration: 4

Found 0 candidate words for rule 'prefix'
Found 0 candidate words for rule 'suffix'
Fou

Finally, let's save our dict to a .txt file.

In [36]:
save_name = "stem_base_words.txt"
save_str = get_x_dict_entries(final_stem_dict)
with open(save_name, 'w') as f:
    f.write(save_str)
print(f"Saved as {save_name}")

Got 2288 entries from input dict
Saved as stem_base_words.txt
